# 03e Ollivier-Ricci Comparison

# Verifying that Ollivier Ricci Works
By reproducing the tutorial cited above...

In [1]:
import networkx as nx
import numpy as np
import math
import importlib
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)
# load GraphRicciCuravture package
from GraphRicciCurvature.OllivierRicci import OllivierRicci
# load python-louvain for modularity computation
import community as community_louvain
# for ARI computation
from sklearn import preprocessing, metrics

In [2]:
G = nx.karate_club_graph()
print(nx.info(G))

Graph named "Zachary's Karate Club" with 34 nodes and 78 edges


/tmp/ipykernel_29246/2350962269.py:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(G))


Computation is achieved via the `OllivierRicci` class, which we feed the graph, and a parameter `alpha`, which determines the "laziness" of the random-walk like distributions used for optimal transport

In [3]:
orc = OllivierRicci(G, alpha=0.5, verbose="TRACE")

In [4]:
orc.compute_ricci_curvature()

TRACE:Number of nodes: 34
TRACE:Number of edges: 78
TRACE:Start to compute all pair shortest path.
TRACE:0.000282 secs for all pair by NetworKit.
INFO:0.382440 secs for Ricci curvature computation.


# On Small (1000 point) Torus

We need to make a Network-X graph out of the adjacency matrix of a standard dataset.

In [5]:
from diffusion_curvature.datasets import torus
from diffusion_curvature.core import DiffusionMatrix

In [6]:
def adaptive_anisotropic_kernel(D, k=10, alpha = 1):
    # Get the distance to the kth neighbor
    distance_to_k_neighbor = np.partition(D,k)[:,k]
    # Populate matrices with this distance for easy division. 
    div1 = np.ones(len(D))[:,None] @ distance_to_k_neighbor[None,:]
    div2 = div1.T
    # compute the gaussian kernel with an adaptive bandwidth
    W = (1/2*np.sqrt(2*np.pi))*(np.exp(-D**2/(2*div1**2))/div1 + np.exp(-D**2/(2*div2**2))/div2)
    # Additional normalization step for density
    D = np.diag(1/(np.sum(W,axis=1)**alpha)) 
    W = D @ W @ D
    return W

In [7]:
X, ks = torus(n=1000)

In [8]:
# compute adjacency matrix
from sklearn.metrics import pairwise_distances
D = pairwise_distances(X)
W = adaptive_anisotropic_kernel(D, k=10, alpha=0.5)

In [9]:
k = 20
threshold = np.mean(np.partition(W,k)[:,-k])
W_binarized = (W > threshold).astype(int)

## Timing Ollivier Ricci

In [10]:
G = nx.from_numpy_array(W_binarized)

In [11]:
G.edges(data=True)

EdgeDataView([(0, 0, {'weight': 1}), (0, 14, {'weight': 1}), (0, 18, {'weight': 1}), (0, 19, {'weight': 1}), (0, 23, {'weight': 1}), (0, 25, {'weight': 1}), (0, 27, {'weight': 1}), (0, 29, {'weight': 1}), (0, 33, {'weight': 1}), (0, 35, {'weight': 1}), (0, 43, {'weight': 1}), (0, 52, {'weight': 1}), (0, 67, {'weight': 1}), (0, 79, {'weight': 1}), (0, 81, {'weight': 1}), (0, 94, {'weight': 1}), (0, 108, {'weight': 1}), (0, 117, {'weight': 1}), (0, 150, {'weight': 1}), (0, 152, {'weight': 1}), (0, 166, {'weight': 1}), (0, 195, {'weight': 1}), (0, 199, {'weight': 1}), (0, 207, {'weight': 1}), (0, 225, {'weight': 1}), (0, 232, {'weight': 1}), (0, 298, {'weight': 1}), (0, 310, {'weight': 1}), (0, 328, {'weight': 1}), (0, 337, {'weight': 1}), (0, 343, {'weight': 1}), (0, 344, {'weight': 1}), (0, 368, {'weight': 1}), (0, 392, {'weight': 1}), (0, 395, {'weight': 1}), (0, 400, {'weight': 1}), (0, 403, {'weight': 1}), (0, 448, {'weight': 1}), (0, 464, {'weight': 1}), (0, 468, {'weight': 1}), (0,

In [12]:
print(nx.info(G))

Graph with 502 nodes and 12560 edges


/tmp/ipykernel_29246/2606185536.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(G))


In [13]:
%%time
orc = OllivierRicci(G, alpha=0.5, verbose="TRACE")

INFO:Self-loop edge detected. Removing 502 self-loop edges.


CPU times: user 43.8 ms, sys: 1.89 ms, total: 45.7 ms
Wall time: 47 ms


In [21]:
%%timeit
orc.compute_ricci_curvature()
G_orc = orc.G.copy()

TRACE:Number of nodes: 502
TRACE:Number of edges: 12058
TRACE:Start to compute all pair shortest path.
TRACE:0.104939 secs for all pair by NetworKit.
INFO:6.843533 secs for Ricci curvature computation.
TRACE:Number of nodes: 502
TRACE:Number of edges: 12058
TRACE:Start to compute all pair shortest path.
TRACE:0.101157 secs for all pair by NetworKit.
INFO:6.880389 secs for Ricci curvature computation.
TRACE:Number of nodes: 502
TRACE:Number of edges: 12058
TRACE:Start to compute all pair shortest path.
TRACE:0.104346 secs for all pair by NetworKit.
INFO:6.870382 secs for Ricci curvature computation.
TRACE:Number of nodes: 502
TRACE:Number of edges: 12058
TRACE:Start to compute all pair shortest path.
TRACE:0.100709 secs for all pair by NetworKit.
INFO:6.899364 secs for Ricci curvature computation.
TRACE:Number of nodes: 502
TRACE:Number of edges: 12058
TRACE:Start to compute all pair shortest path.
TRACE:0.102939 secs for all pair by NetworKit.
INFO:6.882829 secs for Ricci curvature com

7.14 s ± 27.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Diffusion Curvature's Timing

In [18]:
D = np.diag(1/np.sum(W,axis=1))
P = D @ W

In [19]:
from diffusion_curvature.laziness import curvature

In [23]:
%%timeit
diffusion_curvatures = curvature(P, diffusion_powers = 8)

26.8 ms ± 240 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Results

With a torus of 1000 points, Ollivier Ricci has an average time of 7.14 seconds.
Diffusion Curvature does the job in 26.8 milliseconds, making it

In [24]:
7140/26.8

266.4179104477612

266 times faster.

As ORC has to do computations per edge, one would imagine the difference to be *at least* n vs $n^2$. It appears diffusion curvature is actually even faster than that.

# With a large torus

In [26]:
X, ks = torus(n=5000)

In [27]:
# compute adjacency matrix
from sklearn.metrics import pairwise_distances
D = pairwise_distances(X)
W = adaptive_anisotropic_kernel(D, k=10, alpha=0.5)

In [28]:
k = 20
threshold = np.mean(np.partition(W,k)[:,-k])
W_binarized = (W > threshold).astype(int)

## ORC Computation

In [29]:
%%time
orc = OllivierRicci(G, alpha=0.5, verbose="TRACE")

INFO:Self-loop edge detected. Removing 502 self-loop edges.


CPU times: user 47.6 ms, sys: 5.86 ms, total: 53.4 ms
Wall time: 54.8 ms


In [30]:
%%timeit
orc.compute_ricci_curvature()
G_orc = orc.G.copy()

TRACE:Number of nodes: 502
TRACE:Number of edges: 12058
TRACE:Start to compute all pair shortest path.
TRACE:0.105584 secs for all pair by NetworKit.
INFO:7.121563 secs for Ricci curvature computation.
TRACE:Number of nodes: 502
TRACE:Number of edges: 12058
TRACE:Start to compute all pair shortest path.
TRACE:0.100857 secs for all pair by NetworKit.
INFO:7.156031 secs for Ricci curvature computation.
TRACE:Number of nodes: 502
TRACE:Number of edges: 12058
TRACE:Start to compute all pair shortest path.
TRACE:0.102927 secs for all pair by NetworKit.
INFO:7.140787 secs for Ricci curvature computation.
TRACE:Number of nodes: 502
TRACE:Number of edges: 12058
TRACE:Start to compute all pair shortest path.
TRACE:0.104750 secs for all pair by NetworKit.
INFO:7.181922 secs for Ricci curvature computation.
TRACE:Number of nodes: 502
TRACE:Number of edges: 12058
TRACE:Start to compute all pair shortest path.
TRACE:0.102557 secs for all pair by NetworKit.
INFO:7.137960 secs for Ricci curvature com

7.38 s ± 43.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Diffusion Curvature's Timing

In [34]:
D = np.diag(1/np.sum(W_binarized,axis=1))
P = D @ W

In [50]:
from scipy.sparse import csr_array
import scipy
P_sparse = csr_array(P)

In [52]:
P_sparse_powered = P_sparse
for i in range(3):
    P_sparse_powered = P_sparse_powered @ P_sparse_powered

In [38]:
from diffusion_curvature.laziness import curvature

In [39]:
%%timeit
diffusion_curvatures = curvature(P_sparse, diffusion_powers = 8, )

AxisError: axis -1 is out of bounds for array of dimension 0

Now that's interesting, and surprising! On the 5,000 point torus, diffusion curvature goes from being 266 times faster to merely 3 times faster. Why is this? Perhaps sparse matrix multiplications could help recover the difference.

In [45]:
np.partition(P_sparse[0],-3)

NotImplementedError: We have not yet implemented 1D sparse slices; please index using explicit indices, e.g. `x[:, [0]]`

In [53]:
%%time
np.linalg.matrix_power(P,8)

CPU times: user 2.37 s, sys: 45 ms, total: 2.42 s
Wall time: 2.42 s


array([[3.75084383e-18, 7.50491215e-26, 5.89381677e-24, ...,
        1.03553033e-19, 2.04284348e-24, 6.88660646e-24],
       [8.30543611e-26, 6.49051325e-18, 1.38229399e-18, ...,
        3.59041874e-25, 4.08397632e-22, 2.05308461e-19],
       [5.89381677e-24, 1.24906084e-18, 1.81032814e-18, ...,
        2.90111081e-23, 1.16169195e-22, 6.11943856e-20],
       ...,
       [1.21054954e-19, 3.79269585e-25, 3.39143940e-23, ...,
        4.27298971e-18, 3.80877635e-28, 4.64789303e-26],
       [1.94891964e-24, 3.52066924e-22, 1.10828083e-22, ...,
        3.10831173e-28, 3.74317328e-18, 3.07133355e-19],
       [6.56998088e-24, 1.76990052e-19, 5.83808506e-20, ...,
        3.79310811e-26, 3.07133355e-19, 2.28167208e-18]])